In [1]:
import os
import openai
import json

In [2]:
# --- 준비 과정 및 도구 제작 (날씨 요정과 완전히 동일) ---
# client = openai.OpenAI(api_key="여러분의_API_키")
client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [42]:
def get_current_weather(location):
    """지정된 도시의 현재 날씨 정보를 가져오는 가짜 함수입니다."""
    # 이번 시나리오에 더 잘 어울리도록, 날씨를 '비'로 바꿔봅시다.
    print(f"🛠️ [도구 실행] 'get_current_weather' 로봇이 '{location}'의 날씨를 검색합니다.")
    if "서울" in location:
        return json.dumps({"location": "서울", "temperature": "20", "forecast": "비 오는 날"})
    else:
        return json.dumps({"location": location, "temperature": "알 수 없음"})

In [9]:
# --- 에이전트 실행 (핵심 구조는 동일, 질문과 AI의 생각이 달라짐) ---
def run_foodie_agent(user_question):
    print(f"😎 사용자 질문: {user_question}\n")
    
    # [1차 호출] AI는 이 복잡한 질문을 보고, '날씨'라는 핵심 정보를 얻기 위해
    # '날씨 검색 도구'를 써야겠다고 스스로 판단합니다.
    print("🧠 [1차 생각] AI, 이 질문을 해결하려면 어떤 정보가 필요하고, 무슨 도구를 써야 할까?")
    
    messages = [{"role": "user", "content": user_question}]
    tools = [{
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "특정 도시의 현재 날씨를 알려줍니다.",
            "parameters": {
                "type": "object",
                "properties": {"location": {"type": "string", "description": "날씨를 알고 싶은 도시 이름"}},
                "required": ["location"],
            },
        },
    }]
    
    # 이하 1차 호출, 도구 실행, 2차 호출의 코드 구조는 이전 예제들과 동일합니다.
    # 하지만 그 안에 담긴 '의미'는 훨씬 더 고차원적입니다.
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        tools=tools,
        tool_choice="auto",
    )
    
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    
    if tool_calls:
        print("✅ AI의 판단: 먼저 날씨 정보부터 알아봐야겠군!\n")
        tool_call = tool_calls[0]
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)
        
        # [행동] 날씨 검색 도구를 실행하여 '비 오는 날, 20도'라는 정보를 얻습니다.
        
        function_response = get_current_weather(location=function_args.get("location"))
        print(f"📄 [정보 획득] 외부 정보: {function_response}\n")
        
        messages.append(response_message)
        messages.append({
            "tool_call_id": tool_call.id,
            "role": "tool",
            "name": function_name,
            "content": function_response,
        })
        
        # [2차 호출 & 조합] AI는 외부 정보(날씨)와 자신의 내부 지식(음식)을 조합하여 최종 답변을 만듭니다.
        print("🧠 [2차 생각] AI, '비 오는 날'이라는 정보와 너의 지식을 조합해서 메뉴를 추천해줘!")
        second_response = client.chat.completions.create(model="gpt-4o", messages=messages)
        return second_response.choices[0].message.content
    else:
        return response_message.content

In [10]:
# --- 최종 실행 및 성공 확인 ---
final_result = run_foodie_agent("오늘 서울 날씨에 어울리는 저녁 메뉴 추천해줘.")
print("\n🎉 [최종 답변] 맛집 추천가 로봇의 답변입니다!")
print(final_result)

😎 사용자 질문: 오늘 서울 날씨에 어울리는 저녁 메뉴 추천해줘.

🧠 [1차 생각] AI, 이 질문을 해결하려면 어떤 정보가 필요하고, 무슨 도구를 써야 할까?
✅ AI의 판단: 먼저 날씨 정보부터 알아봐야겠군!

🛠️ [도구 실행] 'get_current_weather' 로봇이 '서울'의 날씨를 검색합니다.
📄 [정보 획득] 외부 정보: {"location": "\uc11c\uc6b8", "temperature": "20", "forecast": "\ube44 \uc624\ub294 \ub0a0"}

🧠 [2차 생각] AI, '비 오는 날'이라는 정보와 너의 지식을 조합해서 메뉴를 추천해줘!

🎉 [최종 답변] 맛집 추천가 로봇의 답변입니다!
오늘 서울은 비가 오는 날씨로, 따뜻하고 든든한 저녁 메뉴가 좋을 것 같아요. 불고기 전골이나 김치찌개처럼 뜨겁고 국물이 있는 음식을 추천합니다. 이러한 메뉴들은 비 오는 날 몸을 따뜻하게 해주고 기분도 좋아질 거예요.


---

In [8]:
from rich import print as rprint

In [3]:
user_question="오늘 서울 날씨에 어울리는 저녁 메뉴 추천해줘."

In [5]:
messages = [{"role": "user", "content": user_question}]
messages

[{'role': 'user', 'content': '오늘 서울 날씨에 어울리는 저녁 메뉴 추천해줘.'}]

In [6]:
tools = [{
    "type": "function",
    "function": {
        "name": "get_current_weather",
        "description": "특정 도시의 현재 날씨를 알려줍니다.",
        "parameters": {
            "type": "object",
            "properties": {"location": {"type": "string", "description": "날씨를 알고 싶은 도시 이름"}},
            "required": ["location"],
        },
    },
}]

In [11]:
rprint(tools)

[
    {
        'type': 'function',
        'function': {
            'name': 'get_current_weather',
            'description': '특정 도시의 현재 날씨를 알려줍니다.',
            'parameters': {
                'type': 'object',
                'properties': {'location': {'type': 'string', 'description': '날씨를 알고 싶은 도시 이름'}},
                'required': ['location']
            }
        }
    }
]

In [13]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
    tool_choice="auto",
)

In [23]:
response

ChatCompletion(id='chatcmpl-BuKMccFy98xDznJ6rQxVKCFLGA9t4', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_540320G4ykhuP9waS5WgLaJT', function=Function(arguments='{"location":"서울"}', name='get_current_weather'), type='function')]))], created=1752764346, model='gpt-4o-2024-08-06', object='chat.completion', service_tier='default', system_fingerprint='fp_a288987b44', usage=CompletionUsage(completion_tokens=15, prompt_tokens=74, total_tokens=89, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [15]:
rprint(response)

ChatCompletion(
    id='chatcmpl-BuKMccFy98xDznJ6rQxVKCFLGA9t4',
    choices=[
        Choice(
            finish_reason='tool_calls',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content=None,
                refusal=None,
                role='assistant',
                annotations=[],
                audio=None,
                function_call=None,
                tool_calls=[
                    ChatCompletionMessageToolCall(
                        id='call_540320G4ykhuP9waS5WgLaJT',
                        function=Function(arguments='{"location":"서울"}', name='get_current_weather'),
                        type='function'
                    )
                ]
            )
        )
    ],
    created=1752764346,
    model='gpt-4o-2024-08-06',
    object='chat.completion',
    service_tier='default',
    system_fingerprint='fp_a288987b44',
    usage=CompletionUsage(
        completion_tokens=15,
        prompt_tokens=74,
        total_tokens=89,
        completion_tokens_details=CompletionTokensDetails(
            accepted_prediction_tokens=0,
            audio_tokens=0,
            reasoning_tokens=0,
            rejected_prediction_tokens=0
        ),
        prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)
    )
)

In [30]:
response_message = response.choices[0].message
rprint(response_message)

ChatCompletionMessage(
    content=None,
    refusal=None,
    role='assistant',
    annotations=[],
    audio=None,
    function_call=None,
    tool_calls=[
        ChatCompletionMessageToolCall(
            id='call_540320G4ykhuP9waS5WgLaJT',
            function=Function(arguments='{"location":"서울"}', name='get_current_weather'),
            type='function'
        )
    ]
)

In [32]:
tool_calls = response_message.tool_calls
rprint(tool_calls)


[
    ChatCompletionMessageToolCall(
        id='call_540320G4ykhuP9waS5WgLaJT',
        function=Function(arguments='{"location":"서울"}', name='get_current_weather'),
        type='function'
    )
]

In [35]:
tool_call = tool_calls[0]
rprint(tool_call)

ChatCompletionMessageToolCall(
    id='call_540320G4ykhuP9waS5WgLaJT',
    function=Function(arguments='{"location":"서울"}', name='get_current_weather'),
    type='function'
)

In [37]:
function_name = tool_call.function.name
function_name

'get_current_weather'

In [38]:
tool_call.function.arguments

'{"location":"서울"}'

In [39]:
json.loads(tool_call.function.arguments)

{'location': '서울'}

In [40]:
function_args = json.loads(tool_call.function.arguments)
function_args

{'location': '서울'}

In [43]:
get_current_weather(location=function_args.get("location"))

🛠️ [도구 실행] 'get_current_weather' 로봇이 '서울'의 날씨를 검색합니다.


'{"location": "\\uc11c\\uc6b8", "temperature": "20", "forecast": "\\ube44 \\uc624\\ub294 \\ub0a0"}'

In [46]:
get_current_weather("서울")

🛠️ [도구 실행] 'get_current_weather' 로봇이 '서울'의 날씨를 검색합니다.


'{"location": "\\uc11c\\uc6b8", "temperature": "20", "forecast": "\\ube44 \\uc624\\ub294 \\ub0a0"}'

In [47]:
get_current_weather("부산")

🛠️ [도구 실행] 'get_current_weather' 로봇이 '부산'의 날씨를 검색합니다.


'{"location": "\\ubd80\\uc0b0", "temperature": "\\uc54c \\uc218 \\uc5c6\\uc74c"}'

In [49]:
function_response = get_current_weather(location=function_args.get("location"))
function_response


🛠️ [도구 실행] 'get_current_weather' 로봇이 '서울'의 날씨를 검색합니다.


'{"location": "\\uc11c\\uc6b8", "temperature": "20", "forecast": "\\ube44 \\uc624\\ub294 \\ub0a0"}'

In [52]:
rprint(response_message)

ChatCompletionMessage(
    content=None,
    refusal=None,
    role='assistant',
    annotations=[],
    audio=None,
    function_call=None,
    tool_calls=[
        ChatCompletionMessageToolCall(
            id='call_540320G4ykhuP9waS5WgLaJT',
            function=Function(arguments='{"location":"서울"}', name='get_current_weather'),
            type='function'
        )
    ]
)

In [54]:
messages

[{'role': 'user', 'content': '오늘 서울 날씨에 어울리는 저녁 메뉴 추천해줘.'}]

In [ ]:
# 모델 응답 - messages에 추가
messages.append(response_message)


In [58]:
rprint(messages)

[
    {'role': 'user', 'content': '오늘 서울 날씨에 어울리는 저녁 메뉴 추천해줘.'},
    ChatCompletionMessage(
        content=None,
        refusal=None,
        role='assistant',
        annotations=[],
        audio=None,
        function_call=None,
        tool_calls=[
            ChatCompletionMessageToolCall(
                id='call_540320G4ykhuP9waS5WgLaJT',
                function=Function(arguments='{"location":"서울"}', name='get_current_weather'),
                type='function'
            )
        ]
    )
]

In [60]:
rprint(tool_call)

ChatCompletionMessageToolCall(
    id='call_540320G4ykhuP9waS5WgLaJT',
    function=Function(arguments='{"location":"서울"}', name='get_current_weather'),
    type='function'
)

In [62]:
# tool 실행 결과 - messages에 추가
messages.append({
    "tool_call_id": tool_call.id,
    "role": "tool",
    "name": function_name,
    "content": function_response,
})

In [63]:
rprint(messages)

[
    {'role': 'user', 'content': '오늘 서울 날씨에 어울리는 저녁 메뉴 추천해줘.'},
    ChatCompletionMessage(
        content=None,
        refusal=None,
        role='assistant',
        annotations=[],
        audio=None,
        function_call=None,
        tool_calls=[
            ChatCompletionMessageToolCall(
                id='call_540320G4ykhuP9waS5WgLaJT',
                function=Function(arguments='{"location":"서울"}', name='get_current_weather'),
                type='function'
            )
        ]
    ),
    {
        'tool_call_id': 'call_540320G4ykhuP9waS5WgLaJT',
        'role': 'tool',
        'name': 'get_current_weather',
        'content': '{"location": "\\uc11c\\uc6b8", "temperature": "20", "forecast": "\\ube44 \\uc624\\ub294 
\\ub0a0"}'
    }
]

In [64]:
seconde_response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
)

In [66]:
rprint(seconde_response)

ChatCompletion(
    id='chatcmpl-BuL4xXLyOTpOJQ9FimdiQTqeSdvBL',
    choices=[
        Choice(
            finish_reason='stop',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content='서울의 오늘 날씨가 비가 오는 날씨이므로, 따뜻하고 기분 좋은 저녁 메뉴로 김치찌개나 
불고기를 추천합니다. 국물 요리와 함께라면 비 오는 날씨에도 더욱 분위기 있게 저녁을 즐기실 수 있을 것 같습니다.',
                refusal=None,
                role='assistant',
                annotations=[],
                audio=None,
                function_call=None,
                tool_calls=None
            )
        )
    ],
    created=1752767095,
    model='gpt-4o-2024-08-06',
    object='chat.completion',
    service_tier='default',
    system_fingerprint='fp_07871e2ad8',
    usage=CompletionUsage(
        completion_tokens=63,
        prompt_tokens=82,
        total_tokens=145,
        completion_tokens_details=CompletionTokensDetails(
            accepted_prediction_tokens=0,
            audio_tokens=0,
            reasoning_tokens=0,
            rejected_prediction_tokens=0
        ),
        prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)
    )
)

| 순번 | 역할          | 필드                                | 주의점           |
| -- | ----------- | --------------------------------- | ------------- |
| 1  | `user`      | 사용자 질문                            | 최초 질문         |
| 2  | `assistant` | `tool_calls` 포함된 응답               | 모델이 호출할 함수 지정 |
| 3  | `tool`      | `tool_call_id`, `name`, `content` | 해당 함수 실행 결과   |

이 순서대로 `messages` 에 append 해야 GPT가 이전 context를 정확히 따라가며 응답을 생성합니다.

---